<a href="https://colab.research.google.com/github/abhishekkm8088/Pyspark-Abhi/blob/main/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello world")


hello world
